# Análisis de sentimientos: CV y TFIDF
En este notebook clasificaremos el sentimiento de las noticias de un corpus con 

In [1]:
import pandas as pd
import nltk, re
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words("spanish")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##Corpus Twitter

### Preprocesamiento del texto
Vamos preparar los datos para su uso haciendo una limpieza y preprocesamiento de los mismos. Aplicaremos algunas técnicas que ahora explicaremos.

In [ ]:
def data_preprocess (text, stemming=False, lemmat=True, list_stopwords=None):
    # Eliminamos los signos de puntuación y ponemos el texto en minúscula 
    # eliminando tabulaciones y saltos de línea
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Eliminamos las URLs
    text = re.sub(r'http\S+', '',text)  

    ## Eliminamos los números  
    text = re.sub('[0-9]+', '', text)

    # Eliminamos los emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', text)

    ## Tokenize (convertimos string a lista)
    #list_text = text.split()
    list_text = nltk.word_tokenize(text)

    ## Eliminamos las Stopwords
    if list_stopwords is not None:
        list_text = [word for word in list_text if word not in 
                    list_stopwords]
                
    ## Stemming (convertir palabras en sus raíces)
    if stemming == True:
        ps = nltk.stem.porter.PorterStemmer()
        list_text = [ps.stem(word) for word in list_text]
                
    ## Lemmatization (convertir palabras en su forma canónica)
    if lemmat == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        list_text = [lem.lemmatize(word) for word in list_text]
            
    ## Transformamos la lista a string
    text = " ".join(list_text)
    return text

In [16]:
df = pd.read_csv("data_twitter/tweets_finanzas.csv", delimiter=";")
df

,Tweet,Target,Sent_Target,Sent_Sociedad,Sent_Empresas
0,Repsol empezará a enviar crudo desde Venezuela...,Repsol,POS,POS,POS
1,Tras vetar el petróleo de Moscú todos miran al...,gas,NEG,NEG,NEG
2,"Las demoras de Bruselas, Madrid y Lisboa retra...",luz,NEG,NEG,NEG
3,La gran banca se iguala en rentabilidad a las ...,banca,POS,POS,POS
4,Wall Street reacciona con caídas a los buenos ...,Wall Street,NEG,NEG,NEG
...,...,...,...,...,...
1270,Unicaja Banco obtiene un beneficio neto de 116...,Unicaja Banco,POS,POS,NEG
1271,"Merlin gana 262 millones, un 45% menos, por la...",Merlin,NEG,NEG,POS
1272,Iberdrola se adjudica el mayor número de lotes...,Iberdrola,POS,POS,NEG
1273,CaixaBank lidera venta de fondos de pensiones ...,CaixaBank,POS,POS,NEG


In [ ]:
df["tweet_clean"] = df["Tweet"]

In [ ]:
# Preprocesamos el texto
df["tweet_clean"] = df["Tweet"].apply(lambda x: 
          data_preprocess(x, stemming=False, lemmat=True, 
          list_stopwords=stopwords))
df[["Tweet", "tweet_clean"]]

,Heading,text_clean
0,Apple da el salto al 5G con el iPhone 12,apple da salto g iphone
1,"Bankinter gana 220 millones hasta septiembre, ...",bankinter gana millones septiembre menos
2,Vueling garantiza el empleo de todos sus pilot...,vueling garantiza empleo pilotos final
3,La actividad empresarial de la zona euro se co...,actividad empresarial zona euro contrajo octub...
4,Inditex firma un acuerdo para mantener el empl...,inditex firma acuerdo mantener empleo transfor...
...,...,...
2282,IPSA cierra al alza el día en que Chile gira e...,ipsa cierra alza día chile gira conducción pol...
2283,Profeco suspende gasolineras de Shell por irre...,profeco suspende gasolineras shell irregularid...
2284,Guerra entre los comités de Alu Ibérica por el...,guerra comités alu ibérica acuerdo alcoa
2285,123tinta confía la logística de sus pedidos a MRW,tinta confía logística pedidos mrw


In [ ]:
print(df["Tweet"].iloc[91], " --> ", df["tweet_clean"].iloc[91])

Las empresas ya trasladan a los precios la presión de las materias primas y la energía  -->  empresas trasladan precios presión materias prima energía


In [ ]:
df.to_csv("data_twitter/tweets_finanzas_clean.csv", sep=';', index=False)

In [ ]:
df = pd.read_csv("data_twitter/tweets_finanzas_clean.csv", delimiter=';')

In [ ]:
p_test = 0.20 # Porcentaje de test

random_state= 8
 
from sklearn.model_selection import train_test_split 

# Para poder entrenar es necesario codificar las etiquetas como números. 
# Para eso codificaremos los negativos con 0 y los positivos con 1.
df['Sent_Target'] = df['Sent_Target'].apply(lambda x : 1 if x == 'POS'
                                                      else 0)
df['Sent_Sociedad'] = df['Sent_Sociedad'].apply(lambda x : 1 if x == 'POS'
                                                      else 0)
df['Sent_Empresas'] = df['Sent_Empresas'].apply(lambda x : 1 if x == 'POS'
                                                      else 0)
df = df.sample(frac=1, random_state = random_state)
df_train, df_test = train_test_split (df, test_size = p_test, random_state = random_state)

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para test: ", len(df_test))

Ejemplos usados para entrenar:  1020
Ejemplos usados para test:  255


### CountVectorizer y TF-IDF




Usamos la técnica de bolsas de palabras (BoW) mediante las herramientas CountVectorizer y TfidfVectorizer de la librería Scikit-learn con la que podremos aplicar modelos de aprendizaje automático supervisado como regresión logística, árboles de decisión, random forest y máquinas de vector soporte.


In [ ]:
# Importamos la librerías necesarias
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, f1_score

In [ ]:
# Fijamos el grupo al que se quiere clasificiar el sentimiento: 
# Sent_Target, Sent_Sociedad o Sent_Empresas
col_clasificacion = "Sent_Empresas"
# Fijamos la estrategia de BoW: si aplicamos CV o TF-IDF
use_idf = True

# Creamos el pipeline de CV con LinearSVC
clf_sv = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=use_idf)),
    ('clf', LinearSVC(tol=1e-5, random_state = random_state)),])

params_sv= {'vect__ngram_range': [(1, 1), (1, 2)], 
            'clf__C': [0.1, 0.5, 1.0, 10]}

gs_sv = GridSearchCV(clf_sv, params_sv, scoring='f1', verbose=1)
gs_sv.fit(df_train.tweet_clean, df_train[col_clasificacion])

# Creamos el pipeline de CV con RandomForestClassifier
clf_rf = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=use_idf)),
    ('clf', RandomForestClassifier(n_estimators=200,
                                   random_state = random_state)),])

params_rf= {'vect__ngram_range': [(1, 1), (1, 2)], 
            'clf__min_samples_split':[50,100,250]}

gs_rf = GridSearchCV(clf_rf, params_rf, scoring='f1', verbose=1)
gs_rf.fit(df_train.tweet_clean, df_train[col_clasificacion])

# Creamos el pipeline de CV con LogisticRegression
clf_lr = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=use_idf)),
    ('clf', LogisticRegression(random_state = random_state))])

params_lr= {'vect__ngram_range': [(1, 1), (1, 2)], 
            'clf__C':[.05,.1,.5,1,5]}

gs_lr = GridSearchCV(clf_lr, params_lr, scoring='f1', verbose=1)
gs_lr.fit(df_train.tweet_clean, df_train[col_clasificacion])

# Creamos el pipeline de CV con DecisionTreeClassifier
clf_dt = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=use_idf)),
    ('clf', DecisionTreeClassifier(random_state = random_state))])

params_dt = {'vect__ngram_range': [(1, 1), (1, 2)], 
             'clf__max_leaf_nodes': [2, 4, 6, 8, 10, 15, 20, 30, 40], 
             'clf__min_samples_split': [2, 3, 4]}

gs_dt = GridSearchCV(clf_dt, params_dt, scoring='f1', verbose=1)
gs_dt.fit(df_train.tweet_clean, df_train[col_clasificacion])

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tf', TfidfTransformer()),
                                       ('clf',
                                        DecisionTreeClassifier(random_state=8))]),
             param_grid={'clf__max_leaf_nodes': [2, 4, 6, 8, 10, 15, 20, 30,
                                                 40],
                         'clf__min_samples_split': [2, 3, 4],
                         'vect__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1', verbose=1)

In [ ]:
classifiers = [gs_sv, gs_rf, gs_lr, gs_dt]
name_clf = ["SVC", "Random Forest", "Logistic Regression", "Decision Trees"]
predictors = []
metrics = []

for clf, name in zip(classifiers, name_clf):
    predicted = clf.predict(df_test.tweet_clean)
    recall = recall_score(y_true=df_test[col_clasificacion], y_pred=predicted)
    precision = precision_score(y_true=df_test[col_clasificacion], y_pred=predicted)
    f1 = f1_score(y_true=df_test[col_clasificacion], y_pred=predicted)
    metrics.append([precision, recall, f1])
    predictors.append([name, predicted, metrics])

# Mostramos los resultados
results_CV = pd.DataFrame(columns=["Precision", "Recall", "F1"])
for i in range(4):
  results_CV.loc[i] = metrics[i]

results_CV.index = name_clf
results_CV

,Precision,Recall,F1
SVC,0.505155,0.515789,0.510417
Random Forest,0.553846,0.378947,0.450000
Logistic Regression,0.512500,0.431579,0.468571
Decision Trees,0.448276,0.273684,0.339869


## Corpus de noticias web

### Preprocesamiento del texto
Vamos preparar los datos para su uso haciendo una limpieza y preprocesamiento de los mismos. Aplicaremos algunas técnicas que ahora explicaremos.

In [2]:
def data_preprocess2 (text, stemming=False, lemmat=True, list_stopwords=None):
    # Eliminamos los signos de puntuación y ponemos el texto en minúscula 
    # eliminando tabulaciones y saltos de línea
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Eliminamos las URLs
    text = re.sub(r'http\S+', '',text)  

    ## Eliminamos los números  
    text = re.sub('[0-9]+', '', text)

    ## Tokenize (convertimos string a lista)
    #list_text = text.split()
    list_text = nltk.word_tokenize(text)

    ## Eliminamos las Stopwords
    if list_stopwords is not None:
        list_text = [word for word in list_text if word not in 
                    list_stopwords]
                
    ## Stemming (convertir palabras en sus raíces)
    if stemming == True:
        ps = nltk.stem.porter.PorterStemmer()
        list_text = [ps.stem(word) for word in list_text]
                
    ## Lemmatization (convertir palabras en su forma canónica)
    if lemmat == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        list_text = [lem.lemmatize(word) for word in list_text]
            
    ## Transformamos la lista a string
    text = " ".join(list_text)
    return text

In [3]:
df2 = pd.read_csv("data_web/economy-dataset.csv")
df2 = df2.dropna()

In [ ]:
df2["text_clean"] = df2["Heading"]

In [ ]:
# Preprocesamos el texto
df2["text_clean"] = df2["Heading"].apply(lambda x: 
                                         data_preprocess2(x, stemming=False, 
                                                          lemmat=True,
                                                          list_stopwords=stopwords))
df2[["Heading", "text_clean"]]

,Heading,text_clean
0,Apple da el salto al 5G con el iPhone 12,apple da salto g iphone
1,"Bankinter gana 220 millones hasta septiembre, ...",bankinter gana millones septiembre menos
2,Vueling garantiza el empleo de todos sus pilot...,vueling garantiza empleo pilotos final
3,La actividad empresarial de la zona euro se co...,actividad empresarial zona euro contrajo octub...
4,Inditex firma un acuerdo para mantener el empl...,inditex firma acuerdo mantener empleo transfor...
...,...,...
2282,IPSA cierra al alza el día en que Chile gira e...,ipsa cierra alza día chile gira conducción pol...
2283,Profeco suspende gasolineras de Shell por irre...,profeco suspende gasolineras shell irregularid...
2284,Guerra entre los comités de Alu Ibérica por el...,guerra comités alu ibérica acuerdo alcoa
2285,123tinta confía la logística de sus pedidos a MRW,tinta confía logística pedidos mrw


In [ ]:
print(df2["Heading"].iloc[91], " --> ", df2["text_clean"].iloc[91])

Las empresas ya trasladan a los precios la presión de las materias primas y la energía  -->  empresas trasladan precios presión materias prima energía


In [ ]:
df2.to_csv("data_web/headings_finanzas_clean.csv", index=False)

In [4]:
df2 = pd.read_csv("data_web/headings_finanzas_clean.csv")

In [5]:
p_test = 0.20 # Porcentaje de test

random_state= 8
 
from sklearn.model_selection import train_test_split 

df2 = df2.sample(frac=1, random_state = random_state)
df2_train, df2_test = train_test_split (df2, test_size = p_test, 
                                        random_state = random_state)

print("Ejemplos usados para entrenar: ", len(df2_train))
print("Ejemplos usados para test: ", len(df2_test))

Ejemplos usados para entrenar:  1816
Ejemplos usados para test:  454


### CountVectorizer y TF-IDF




Usamos la técnica de bolsas de palabras (BoW) mediante las herramientas CountVectorizer y TfidfVectorizer de la librería Scikit-learn con la que podremos aplicar modelos de aprendizaje automático supervisado como regresión logística, árboles de decisión, random forest y máquinas de vector soporte.


In [6]:
# Importamos la librerías necesarias
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, f1_score

In [11]:
# Fijamos el grupo al que se quiere clasificiar el sentimiento: 
# S. Target, S. Society o S. Others
col_clasificacion = "S. Target"
# Fijamos la estrategia de BoW: si aplicamos CV o TF-IDF
use_idf = False

# Creamos el pipeline de CV con LinearSVC
clf_sv = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=use_idf)),
    ('clf', LinearSVC(tol=1e-5, random_state = random_state)),])

params_sv= {'vect__ngram_range': [(1, 1), (1, 2)], 
            'clf__C': [0.1, 0.5, 1.0, 10]}

gs_sv = GridSearchCV(clf_sv, params_sv, verbose=1)
gs_sv.fit(df2_train.text_clean, df2_train[col_clasificacion])

# Creamos el pipeline de CV con RandomForestClassifier
clf_rf = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=use_idf)),
    ('clf', RandomForestClassifier(n_estimators=200,
                                   random_state = random_state)),])

params_rf= {'vect__ngram_range': [(1, 1), (1, 2)], 
            'clf__min_samples_split':[50,100,250]}

gs_rf = GridSearchCV(clf_rf, params_rf, verbose=1)
gs_rf.fit(df2_train.text_clean, df2_train[col_clasificacion])

# Creamos el pipeline de CV con LogisticRegression
clf_lr = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=use_idf)),
    ('clf', LogisticRegression(random_state = random_state, 
                               max_iter=10000,
                               multi_class='multinomial'))])

params_lr= {'vect__ngram_range': [(1, 1), (1, 2)], 
            'clf__C':[.05,.1,.5,1,5]}

gs_lr = GridSearchCV(clf_lr, params_lr, verbose=1)
gs_lr.fit(df2_train.text_clean, df2_train[col_clasificacion])

# Creamos el pipeline de CV con DecisionTreeClassifier
clf_dt = Pipeline([
    ('vect', CountVectorizer()),
    ('tf', TfidfTransformer(use_idf=use_idf)),
    ('clf', DecisionTreeClassifier(random_state = random_state))])

params_dt = {'vect__ngram_range': [(1, 1), (1, 2)], 
             'clf__max_leaf_nodes': [2, 4, 6, 8, 10, 15, 20, 30, 40], 
             'clf__min_samples_split': [2, 3, 4]}

gs_dt = GridSearchCV(clf_dt, params_dt, verbose=1)
gs_dt.fit(df2_train.text_clean, df2_train[col_clasificacion])

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tf', TfidfTransformer(use_idf=False)),
                                       ('clf',
                                        DecisionTreeClassifier(random_state=8))]),
             param_grid={'clf__max_leaf_nodes': [2, 4, 6, 8, 10, 15, 20, 30,
                                                 40],
                         'clf__min_samples_split': [2, 3, 4],
                         'vect__ngram_range': [(1, 1), (1, 2)]},
             verbose=1)

In [12]:
classifiers = [gs_sv, gs_rf, gs_lr, gs_dt]

name_clf = ["SVC", "Random Forest", "Logistic Regression", "Decision Trees"]
predictors = []
metrics = []

for clf, name in zip(classifiers, name_clf):
    predicted = clf.predict(df2_test.text_clean)
    recall = recall_score(y_true=df2_test[col_clasificacion], y_pred=predicted, average="weighted")
    precision = precision_score(y_true=df2_test[col_clasificacion], y_pred=predicted, average="weighted")
    f1 = f1_score(y_true=df2_test[col_clasificacion], y_pred=predicted, average="weighted")
    metrics.append([precision, recall, f1])
    predictors.append([name, predicted, metrics])

# Mostramos los resultados
results_CV = pd.DataFrame(columns=["Precision", "Recall", "F1"])
for i in range(4):
  results_CV.loc[i] = metrics[i]

results_CV.index = name_clf
results_CV

,Precision,Recall,F1
SVC,0.635100,0.662996,0.623218
Random Forest,0.651724,0.649780,0.576818
Logistic Regression,0.652668,0.678414,0.638770
Decision Trees,0.621059,0.658590,0.595746
